In [1]:
# 이미지 별 x, y 좌표 추출
def extract_coordinates(dataframe):
    coordinates = {}
    for image_name in dataframe['image_name'].unique():
        # 해당 이미지의 데이터 필터링
        image_data = dataframe[dataframe['image_name'] == image_name]
        # x, y 좌표 배열 생성
        x_coords = image_data['x'].tolist()
        y_coords = image_data['y'].tolist()
        # 딕셔너리에 저장
        coordinates[image_name] = {'x': x_coords, 'y': y_coords}
    return coordinates

In [2]:
import numpy as np
import pandas as pd

neutral_df = pd.read_csv("../output/neutral.csv")
angry_df = pd.read_csv("../output/angry.csv")
happy_df = pd.read_csv("../output/happy.csv")
neutral_df['label'] = 0
happy_df['label'] = 1
angry_df['label'] = 2

face_metadata_df = pd.concat([neutral_df, angry_df, happy_df], ignore_index=True)
face_metadata_df = face_metadata_df.sort_values(by="image_name")

In [3]:
face_metadata_df.head(5)

,image_name,face_id,point_id,x,y,label
0,C0000.jpg,0,0,152,106,0
36,C0000.jpg,0,36,159,103,0
37,C0000.jpg,0,37,161,101,0
38,C0000.jpg,0,38,165,101,0
39,C0000.jpg,0,39,169,104,0


In [4]:
x_face_metadata_df = face_metadata_df.drop('label', axis=1)
y_face_metadata_df = face_metadata_df[['image_name', 'label']]

In [5]:
x_face_metadata_df.head(5)

,image_name,face_id,point_id,x,y
0,C0000.jpg,0,0,152,106
36,C0000.jpg,0,36,159,103
37,C0000.jpg,0,37,161,101
38,C0000.jpg,0,38,165,101
39,C0000.jpg,0,39,169,104


In [6]:
y_face_metadata_df.head(5)

,image_name,label
0,C0000.jpg,0
36,C0000.jpg,0
37,C0000.jpg,0
38,C0000.jpg,0
39,C0000.jpg,0


In [7]:
from sklearn.model_selection import train_test_split

def convert_to_X_train(target_df: pd.DataFrame):
    result = list()
    image_names = (target_df["image_name"]).unique()

    for image_name in image_names:
        points = target_df[target_df["image_name"] == image_name][["x", "y"]].values
        result.append(points)
    
    return np.array(result)
    
x_train_data = convert_to_X_train(x_face_metadata_df)

In [8]:
def convert_to_Y_train(target_df: pd.DataFrame):
    result = target_df.drop_duplicates('image_name').copy()
    result['neutral'] = result['label'].map({ 0: 1, 1: 0, 2: 0})
    result['happy'] = result['label'].map({ 0: 0, 1: 1, 2: 0 })
    result['angry'] = result['label'].map({ 0: 0, 1: 0, 2: 1 })                                           
    
    return np.array(result.drop(['image_name', 'label'], axis=1))

y_train_data = convert_to_Y_train(y_face_metadata_df)

In [9]:
X_train, X_test, Y_train, Y_test = train_test_split(x_train_data,
                                                    y_train_data,
                                                    test_size=0.2,
                                                    random_state=11)

In [10]:
print(X_train.shape[0])
print(Y_train.shape[0])

7200
7200


In [11]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Conv1D, Dropout, BatchNormalization, GlobalAveragePooling1D
from tensorflow.keras.callbacks import Callback, ReduceLROnPlateau

In [ ]:
model = Sequential([
    Conv1D(64, kernel_size=3, activation='relu', input_shape=(68, 2)),
    BatchNormalization(),  # 모델 안정화
    Dense(32 , activation='relu'),
    Dense(32 , activation='relu'),
    Dense(64 , activation='relu'),
    Dense(128, activation='relu'), 
    Dense(256, activation='relu'), Dropout(0.5),
    Dense(256, activation='relu'), Dropout(0.5),  
    Dense(128, activation='relu'),
    Dense(64 , activation='relu'),
    Dense(32 , activation='relu'),
    Dense(32 , activation='relu'),
    
    GlobalAveragePooling1D(),
    Dense(3, activation='softmax') # 출력층
])

c:\Users\daheung\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [38]:
from model_analyzer import NEpochCallback, NBatchCallback

In [39]:
batch_callback = NBatchCallback(n=10)
epoch_callback = NEpochCallback(n= 2, filepath="./weights/")

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6)
model.compile(optimizer='nadam', loss='categorical_crossentropy', metrics=['accuracy'])

In [40]:
model.fit(X_train, Y_train, epochs=10, batch_size=128, callbacks=[batch_callback, epoch_callback])

Epoch 1/10
 5/57 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.3032 - loss: 1.1017Batch 10: logs={'accuracy': 0.34062498807907104, 'loss': 1.0993856191635132}
17/57 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.3231 - loss: 1.1003Batch 20: logs={'accuracy': 0.3246093690395355, 'loss': 1.099638819694519}
26/57 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.3254 - loss: 1.1001Batch 30: logs={'accuracy': 0.3356770873069763, 'loss': 1.0991946458816528}
36/57 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.3282 - loss: 1.0998Batch 40: logs={'accuracy': 0.33984375, 'loss': 1.0987132787704468}
47/57 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.3305 - loss: 1.0995Batch 50: logs={'accuracy': 0.3400000035762787, 'loss': 1.0985231399536133}
57/57 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - accuracy: 0.3320 - loss: 1.0993
Epoch 2/10
 1/57 ━━━━━━━━━━━━━━━━━━━━ 1s 28ms/step - accuracy: 0.3906 - loss: 1.0934Batch 60: logs={'accuracy': 0.3880208432674408, 'loss': 1.0943177938461304}
12/57 ━━━━━━━━━━━━

In [41]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score, precision_score, recall_score

In [42]:
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(Y_test, axis=1)

accuracy = accuracy_score(y_true, y_pred_classes)
precision = precision_score(y_true, y_pred_classes, average='weighted')
recall = recall_score(y_true, y_pred_classes, average='weighted')
f1 = f1_score(y_true, y_pred_classes, average='weighted')

print(f"Accuracy: {accuracy}, Precision: {precision}, Recall: {recall}, F1 Score: {f1}")

57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
Accuracy: 0.3488888888888889, Precision: 0.20541221603067242, Recall: 0.3488888888888889, F1 Score: 0.18978981359649127


c:\Users\daheung\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
